# LaZerRanks Glicko

In [1]:
using JuliaDB, FileIO, IterableTables, CSVFiles, CSV;|

  likely near In[1]:3
  likely near In[1]:6
  likely near In[1]:7
  likely near In[1]:10


| (generic function with 50 methods)

In [2]:
function load_elapsed_table()
    path_comps = "WCA_export_Competitions.tsv"
    wca_comps = loadndsparse(path_comps; delim = '\t', quotechar = '\\',
        colparsers=Dict("id"=>String, "name"=>String, "year"=>Int, 
            "month"=>Int, "day"=>Int, "endMonth"=>Int, "endDay"=>Int))
    function comp_date(c)
        Date(c[6], c[7], c[8])
    end
    function days_elapsed(comp)
         Date(now()) - comp_date(comp)
    end
    comp_ids = []
    comp_elapsed = []
    for i in 1:length(wca_comps)
        c = wca_comps[i]
        push!(comp_ids, c[1])
        push!(comp_elapsed, days_elapsed(c).value)
    end
    
    elapsed_table = table(comp_ids, comp_elapsed, names=[:competitionId, :elapsed], pkey = :elapsed)
    return elapsed_table
end

load_elapsed_table (generic function with 1 method)

In [3]:
function load_elapsed_sparse()
    table = load_elapsed_table()
    comp_ids = []
    comp_elapsed = []
    for i in 1:length(table)
        c = table[i]
        push!(comp_ids, c[1])
        push!(comp_elapsed, c[2])
    end
    elapsed_sparse = ndsparse(@NT(competitionId=comp_ids), comp_elapsed)
end    

load_elapsed_sparse (generic function with 1 method)

In [4]:
elapsed_table = load_elapsed_table()
elapsed_sparse = load_elapsed_sparse()
    
results = loadndsparse("WCA_export_Results.tsv"; delim = '\t',
      indexcols = [:competitionId, :eventId, :roundTypeId, :personId],
      colparsers=Dict("personId"=>String,"eventId"=>String,"competitionId"=>String, "roundTypeId"=>String, "pos" =>Int))

4-d NDSparse with 1965270 values (13 field named tuples):
    Dimensions
#  colname        type
────────────────────────
1  competitionId  String
2  eventId        String
3  roundTypeId    String
4  personId       String
    Values
#   colname                type
─────────────────────────────────
5   pos                    Int64
6   best                   Int64
7   average                Int64
8   personName             String
9   personCountryId        String
10  formatId               String
11  value1                 Int64
12  value2                 Int64
13  value3                 Int64
14  value4                 Int64
15  value5                 Int64
16  regionalSingleRecord   String
17  regionalAverageRecord  String

In [5]:
function genGlickoArray(comp, event, round)
    
    myresults = table(results[comp, event, round, :])
    mytable = table(select(myresults, (:personId, :pos)), pkey=:pos)
    
    return mytable
end
    

genGlickoArray (generic function with 1 method)

In [6]:
function genAllGlickoArrays(event)
    comps = []
    for i in 1:length(elapsed_table)
        push!(comps, elapsed_table[i][1])
    end
    comps = reverse(comps)
    allGlickoArrays = []
    lengthOfComps = length(comps)
    for i in 1:length(comps)
        push!(allGlickoArrays, genGlickoArray(comps[i], event, "0"))
        push!(allGlickoArrays, genGlickoArray(comps[i], event, "1"))
        push!(allGlickoArrays, genGlickoArray(comps[i], event, "2"))
        push!(allGlickoArrays, genGlickoArray(comps[i], event, "3"))
        push!(allGlickoArrays, genGlickoArray(comps[i], event, "b"))
        push!(allGlickoArrays, genGlickoArray(comps[i], event, "c"))
        push!(allGlickoArrays, genGlickoArray(comps[i], event, "d"))
        push!(allGlickoArrays, genGlickoArray(comps[i], event, "e"))
        push!(allGlickoArrays, genGlickoArray(comps[i], event, "f"))
        push!(allGlickoArrays, genGlickoArray(comps[i], event, "g"))
        push!(allGlickoArrays, genGlickoArray(comps[i], event, "h"))
        print("\r$i / $lengthOfComps Comps")
    end
    output = []
    for i in 1:length(allGlickoArrays)
        if length(allGlickoArrays[i]) != 0
            push!(output, allGlickoArrays[i])
        end
    end
    return output
end

genAllGlickoArrays (generic function with 1 method)

In [70]:
function findSigma(mu, phi, sigma, change, v)
    tau = 0.5
    epsilon = 0.001
    alpha = log(sigma^2)
    
    function f(x)
        tmp = phi^2 + v + e^x
        a = (e^x)*(change^2 - tmp)/(2*tmp^2)
        b = (x - alpha) / (tau^2)
        return a - b
    end
    
    a = alpha
    if change^2 > phi^2 + v
        b = log(change^2 - phi^2 - v)
    else
        k = 1
        while f(alpha - k*tau) < 0
            k += 1
        end
        b = alpha - k*tau 
    end
    fa = f(a)
    fb = f(b)
    while abs(b - a) > epsilon
        c = a + ((a - b)*fa)/(fb - fa)
        fc = f(c)
        if fc*fb < 0 
            a = b
            fa = fb
        else
            fa /= 2
        end
        b = c
        fb = fc
    end
    return e^(a/2)
end

findSigma (generic function with 1 method)

In [9]:
path_persons = "WCA_export_Persons.tsv"
wca_persons = loadtable(path_persons; delim = '\t', quotechar = '\\',
    colparsers=Dict("id"=>String,"subid"=>Int, "name"=>String, "CountryId"=>String, "gender"=>String));

persons = loadndsparse(path_persons; delim = '\t', quotechar = '\\', indexcols = [:id, :subid],
    colparsers=Dict("id"=>String,"subid"=>Int, "name"=>String, "CountryId"=>String, "gender"=>String));

In [83]:
function generateRatings(thing) 
    playerids = []
    for i in 1:length(wca_persons)
        push!(playerids, wca_persons[i][1])
    end

    #reset ratings
    ratings = []
    RDs = []
    vols = []
    for i in 1:length(playerids)
        push!(ratings, 1500.0)
        push!(RDs, 25.0)
        push!(vols, 0.01)
    end

    ratingdict = Dict(zip(playerids, ratings))
    RDdict = Dict(zip(playerids, RDs))
    voldict = Dict(zip(playerids, vols));
    
    for i in 1:length(thing)
        calcRatings(thing[i], ratingdict, RDdict, voldict)
        print("\r$i / ", length(thing))
    end
    
    RatingList = sort(collect(zip(values(ratingdict),keys(ratingdict))), rev = true)
    
    htb = @sprintf "%4s %-30s \t %-7s %-3s %-15s" "Rank" "Name" "Rating" "RD" "Citizen of"
    println()
    println(htb)
    for i in 1:length(RatingList)
        id = RatingList[i][2]
        name = persons[id, 1].name
        if length(name) > 30
            name = name[1:30]"..."
        end
        rating = RatingList[i][1]
        RD = RDdict[id]
        country = persons[id, 1].countryId
        tb = @sprintf "%4d %30s \t %6.2f %-3d %-15s" i  rpad(name, 30) rating RD rpad(country, 15)
        println(tb)
    end
    return RatingList
end

generateRatings (generic function with 1 method)

In [166]:
function calcRatings(data, ratingdict, RDdict, voldict)
    tau = 0.5
    epsilon = 0.001
    maxsize = 186
    maxmulti = 0.272
    multislope = 0.00391
    conv = 173.7178
    
    g(phi) = 1/sqrt(1 + 3*phi^2/pi^2)
    E(mu, OppMu, OppPhi) = 1/(1 + exp(-g(oppPhi)*(mu - oppMu)))
    
    for i in 1:length(data), j in 1:length(data)
        mu = (ratingdict[data[i].personId] - 1500.0)/conv
        phi = (RDdict[data[i].personId])/conv
        sigma = voldict[data[i].personId]
        v_inv = 0
        change = 0
        if i != j
            oppMu = (ratingdict[data[j].personId] - 1500.0)/conv
            oppPhi = RDdict[data[j].personId]/conv
            if data[i].pos > data[j].pos
                S = 0.0
            elseif data[i].pos < data[j].pos
                S = 1.0
            else
                S = 0.5
            end

            v_inv += g(oppPhi)^2*E(mu, OppMu, OppPhi)*(1-E(mu, OppMu, OppPhi))
            change += g(oppPhi)*(S - E(mu, OppMu, OppPhi))
        end
        if v_inv != 0
            v = 1/v_inv
        else
            v = 0
        end
        delta = v*change
        newSigma = findSigma(mu, phi, sigma, change, v)
        phiAst = sqrt(phi^2 + newSigma^2)
        newPhi = 1/sqrt(1/(phiAst^2) + 1/v)
        newMu = mu + (newPhi^2)*change
        ratingdict[data[i].personId] = newMu*conv + 1500.0
        RDdict[data[i].personId] = newPhi*conv
        voldict[data[i].personId] = newSigma
    end
end

calcRatings (generic function with 2 methods)

In [20]:
thing = genAllGlickoArrays("333");

5566 / 5566 Comps

In [85]:
ratings = generateRatings(thing) # 3x3 Cube

14928 / 14928
Rank Name                           	 Rating  RD  Citizen of     
   1 Demid Zyryanov (Демид Зырянов) 	    NaN NaN Russia         
   2 Arkadiusz Żynel                	    NaN NaN Poland         
   3 Tomasz Żymuła                  	    NaN NaN Poland         
   4 Celina Zymula                  	    NaN NaN Poland         
   5 Agata Żymuła                   	    NaN NaN Poland         
   6 Alexander Zykov (Алексан...    	    NaN NaN Russia         
   7 Anna Zybunova (Анна Зибунова)  	    NaN NaN Ukraine        
   8 Kylie Zwiers                   	    NaN NaN Canada         
   9 Matthias Zwicky                	    NaN NaN Switzerland    
  10 Ben Zwicky                     	    NaN NaN Switzerland    
  11 Andrew Zwaagstra               	    NaN NaN Canada         
  12 Henry Vander Zwaag             	    NaN NaN USA            
  13 Aleksei Zvorygin               	    NaN NaN Russia         
  14 Nikita Zvonkov (Никита З...    	    NaN NaN Russia         
  15 Dmitr

 149 Alisher Zhuman (Алишер Жуман)  	    NaN NaN Kazakhstan     
 150 Lucas Zhu                      	    NaN NaN France         
 151 Longfei Zhu (祝龙飞)           	    NaN NaN China          
 152 Lixin Zhu (朱立新)             	    NaN NaN China          
 153 Kaihua Zhu                     	    NaN NaN New Zealand    
 154 Alexey Zhuk (Алексей Жук)      	    NaN NaN Belarus        
 155 Vyacheslav Zhukov (Вячесл...   	    NaN NaN Russia         
 156 Kevin Zhu                      	    NaN NaN Australia      
 157 Andrei Zhukov (Андрей Жуков)   	    NaN NaN Belarus        
 158 Jingyu Zhu (朱靖予)            	    NaN NaN China          
 159 Junping Zhu (朱俊凭)           	    NaN NaN China          
 160 Joshua Zhu                     	    NaN NaN Canada         
 161 Jianming Zhu (朱建铭)          	    NaN NaN China          
 162 Jianzhen Zhu (朱建震)          	    NaN NaN China          
 163 Jia Zhu (朱加)                 	    NaN NaN China          
 164 Jiaqing Zhu (朱佳庆)           	    NaN NaN

 301 Pavel Zherebtsov (Павел Ж...   	    NaN NaN Russia         
 302 Ooi Ee Zhe                     	    NaN NaN Malaysia       
 303 Keran Zheng (郑科冉)           	    NaN NaN China          
 304 Enze Zheng (郑恩泽)            	    NaN NaN China          
 305 Mai Zheng (郑迈)               	    NaN NaN China          
 306 Yuqiao Zheng (郑渝桥)          	    NaN NaN China          
 307 Dongsheng Zheng (郑东升)       	    NaN NaN China          
 308 Yaning Zheng (郑雅宁)          	    NaN NaN China          
 309 Saisai Zheng (郑赛赛)          	    NaN NaN China          
 310 Kunchen Zheng (郑鲲宸)         	    NaN NaN China          
 311 Jun Zheng (郑骏)               	    NaN NaN China          
 312 Zihao Zheng (郑梓浩)           	    NaN NaN China          
 313 Yuqi Zheng (郑雨琪)            	    NaN NaN China          
 314 Bowen Zheng (郑博文)           	    NaN NaN China          
 315 Xinping Zheng (郑馨平)         	    NaN NaN China          
 316 Jiaming Zheng (郑佳明)         	    NaN NaN China          


 453 William Zhang                  	    NaN NaN Canada         
 454 Wenjing Zhang (张文静)         	    NaN NaN China          
 455 Wan Zhang (张宛)               	    NaN NaN China          
 456 Weilin Zhang (张蔚林)          	    NaN NaN China          
 457 Wenxi Zhang (张雯熙)           	    NaN NaN China          
 458 Wenjing Zhang (张雯菁)         	    NaN NaN China          
 459 Wenxin Zhang (张文馨)          	    NaN NaN China          
 460 Wangzi Zhan (占王梓)           	    NaN NaN China          
 461 Wenxin Zhang (张文欣)          	    NaN NaN China          
 462 Andrii Zhavoronkov (Андрі...   	    NaN NaN Ukraine        
 463 Artem Zhavoronkov              	    NaN NaN Kazakhstan     
 464 Hatt Dylan Z.                  	    NaN NaN Canada         
 465 Tianqi Zhang (张天祺)          	    NaN NaN China          
 466 Tongfei Zhang (张茼霏)         	    NaN NaN China          
 467 Tim Zhang                      	    NaN NaN China          
 468 Tengyu Zhang (张腾誉)          	    NaN NaN China   

 604 Yisheng Zhang (张懿圣)         	    NaN NaN China          
 605 Yichao Zhang (张艺超)          	    NaN NaN China          
 606 Xiyue Zhang (张希岳)           	    NaN NaN China          
 607 Xiaotian Zhang (张晓天)        	    NaN NaN China          
 608 Xiaoming Zhang (张笑铭)        	    NaN NaN China          
 609 Weiyi Zhang (张炜祎)           	    NaN NaN China          
 610 Minghao Zhang (张明昊)         	    NaN NaN China          
 611 Kaishuo Zhang (张闿铄)         	    NaN NaN China          
 612 Jiaming Zhang (张嘉铭)         	    NaN NaN China          
 613 Haozhuo Zhang (张昊卓)         	    NaN NaN China          
 614 Boyang Zhang (张博洋)          	    NaN NaN China          
 615 Rujiu Zhang (张如九)           	    NaN NaN China          
 616 Enrui Zhang (张恩瑞)           	    NaN NaN China          
 617 Canhui Zhang (张灿辉)          	    NaN NaN China          
 618 Zhengyi Zhang (张政一)         	    NaN NaN China          
 619 Yibo Zhang (张逸博)            	    NaN NaN China          
 620 Xin

...

4455 Xun Sun (孙逊)                 	    NaN NaN China          
4456 Xinghao Sun (孙星昊)           	    NaN NaN China          
4457 Wenlong Sun (孙文龙)           	    NaN NaN China          
4458 Wenyi Sun (孙文怡)             	    NaN NaN China          
4459 Wei-Chieh Sun (孫韋傑)         	    NaN NaN Taiwan         
4460 Wei Sun (孙巍)                 	    NaN NaN China          
4461 Wenjun Sun (孙闻骏)            	    NaN NaN China          
4462 Shengrong Sun (孙圣容)         	    NaN NaN China          
4463 Rui Sun (孙睿)                 	    NaN NaN China          
4464 Ruonan Sun (孙若楠)            	    NaN NaN China          
4465 Rui Sun (孙睿)                 	    NaN NaN China          
4466 Ruiyang Sun (孙睿阳)           	    NaN NaN China          
4467 Ruoxi Sun (孙若曦)             	    NaN NaN China          
4468 Ruiyang Sun (孙睿阳)           	    NaN NaN China          
4469 Ruixuan Sun (孙睿轩)           	    NaN NaN China          
4470 Quanrui Sun (孙全瑞)           	    NaN NaN China          
4471

4608 Navaj Subash                   	    NaN NaN India          
4609 Luka Subari                    	    NaN NaN Georgia        
4610 Alejandro Rivera Suaza         	    NaN NaN Colombia       
4611 Manuel Felipe Nieto Suarez     	    NaN NaN Colombia       
4612 Estefania Suarez-Fernandez     	    NaN NaN USA            
4613 Marcos Fernandez Suarez        	    NaN NaN Spain          
4614 Frankie Dejay Suarez           	    NaN NaN Philippines    
4615 Antonio Adorno Suarez          	    NaN NaN Bolivia        
4616 Rocio Suarez                   	    NaN NaN Argentina      
4617 Pilar Suarez                   	    NaN NaN Argentina      
4618 Heider Suarez                  	    NaN NaN Colombia       
4619 Radmir Stusik (Радмир Стусик)  	    NaN NaN Kazakhstan     
4620 Brody Sturgis                  	    NaN NaN USA            
4621 Steffi Stumpos                 	    NaN NaN USA            
4622 Nathan Stumpf                  	    NaN NaN USA            
4623 Adeline Stults      

Excessive output truncated after 524348 bytes.


4763 David Steele                   	    NaN NaN USA            
4764 Eli Steele                     	    NaN NaN USA            
4765 Henry Steel                    	    NaN NaN USA            
4766 Ryan Steer                     	    NaN NaN USA            
4767 Ashton Steed                   	    NaN NaN USA            
4768 Kaden Lucas Steeves            	    NaN NaN Canada         
4769 Ivan Steblynskyi (Іван Ст...   	    NaN NaN Ukraine        
4770 Daniel Stead                   	    NaN NaN New Zealand    
4771 Tanner Steah                   	    NaN NaN USA            
4772 Iason Adrianos Stavropoulos    	    NaN NaN Greece         
4773 Santiago Jeremías Staude       	    NaN NaN Argentina      
4774 Cillian Staunton               	    NaN NaN Ireland        
4775 Maxim Stativko (Максим С...    	    NaN NaN Russia         
4776 Grant Staten                   	    NaN NaN USA            
4777 Vasilis Stathopoulos           	    NaN NaN Greece         
4778 Herman Statsko (Гер

116190-element Array{Tuple{Any,Any},1}:
 (NaN, "2018ZYRY01")     
 (NaN, "2018ZYNE01")     
 (NaN, "2018ZYMU03")     
 (NaN, "2018ZYMU02")     
 (NaN, "2018ZYMU01")     
 (NaN, "2018ZYKO01")     
 (NaN, "2018ZYBU01")     
 (NaN, "2018ZWIE01")     
 (NaN, "2018ZWIC02")     
 (NaN, "2018ZWIC01")     
 (NaN, "2018ZWAA02")     
 (NaN, "2018ZWAA01")     
 (NaN, "2018ZVOR01")     
 ⋮                       
 (997.186, "2005MINO02") 
 (984.798, "1982TRAJ01") 
 (975.783, "2004PERE01") 
 (890.311, "2005FEDE01") 
 (803.788, "2005SING01") 
 (778.788, "2004MATI02") 
 (774.383, "2006TSAI02") 
 (615.856, "2004ZYBU01") 
 (349.92, "2004HENA01")  
 (298.566, "2009LERT01") 
 (-419.355, "2003POUR01")
 (-1413.69, "2009SUTH01")

In [14]:
sum = 0
for i in 1:length(ratings)
    sum += ratings[i][1]
end
println("mean: ", sum/length(ratings))
println("median: ", ratings[Int.(length(ratings)/2)][1])
println("min: ", ratings[length(ratings)][1])
println("max: ", ratings[1][1])

LoadError: [91mUndefVarError: ratings not defined[39m

In [120]:
calcRatings(thing[2])

In [168]:
#reset ratings
    ratings = []
    RDs = []
    vols = []
    for i in 1:length(playerids)
        push!(ratings, 1500.0)
        push!(RDs, 25.0)
        push!(vols, 0.01)
    end
ratingdict = Dict(zip(playerids, ratings))
    RDdict = Dict(zip(playerids, RDs))
    voldict = Dict(zip(playerids, vols));

In [187]:
data

Table with 83 rows, 2 columns:
personId      pos
─────────────────
"2003KNIG01"  1
"2003AKIM01"  2
"2003ALLE01"  3
"2003VAND01"  4
"1982FRID01"  5
"2003BOND01"  6
"2003MAKI01"  7
"2003LIDO01"  8
"2003CAMA01"  9
"2003KONI01"  10
"2003BRUC01"  11
"2003WESL01"  12
⋮
"2003GOOD01"  73
"2003BRAN03"  74
"2003KNIG02"  75
"2003ALGA01"  76
"2003THOM01"  77
"2003JOZW01"  78
"2003STAU01"  79
"2003TING01"  80
"2003ZBOR01"  81
"2003GRAN01"  82
"2003POUR01"  83

In [188]:
tau = 0.5
epsilon = 0.001
maxsize = 186
maxmulti = 0.272
multislope = 0.00391
conv = 173.7178

g(phi) = 1/sqrt(1 + 3*phi^2/pi^2)
E(mu, oppMu, oppPhi) = 1/(1 + exp(-g(oppPhi)*(mu - oppMu)))

updateRatings = []
updateRDs = []
updateVols = []

for i in 1:length(data)
    mu = (ratingdict[data[i].personId] - 1500.0)/conv
    phi = (RDdict[data[i].personId])/conv
    sigma = voldict[data[i].personId]
    v_inv = 0
    change = 0
    for j in 1:length(data)
        if i != j
            oppMu = (ratingdict[data[j].personId] - 1500.0)/conv
            oppPhi = RDdict[data[j].personId]/conv
            if data[i].pos > data[j].pos
                S = 0.0
            elseif data[i].pos < data[j].pos
                S = 1.0
            else
                S = 0.5
            end

            v_inv += g(oppPhi)^2*E(mu, oppMu, oppPhi)*(1-E(mu, oppMu, oppPhi))
            change += g(oppPhi)*(S - E(mu, oppMu, oppPhi))
        end
    end

    if v_inv != 0
        v = 1/v_inv
    else
        v = 0
    end

    println("v: ", v)
    println("change: ", change)
    delta = v*change
    newSigma = findSigma(mu, phi, sigma, change, v)
    phiAst = sqrt(phi^2 + newSigma^2)
    newPhi = 1/sqrt(1/(phiAst^2) + 1/v)
    newMu = mu + (newPhi^2)*change
    newRating = newMu*conv + 1500.0
    newRD = newPhi*conv

    println("new rating: ", newRating)
    println("new RD: ", newRD)
    println("new vol: ", newSigma)
    push!(updateRatings, newRating)
    push!(updateRDs, newRD)
    push!(updateVols, newSigma)
end
for i in 1:length(data)
    ratingdict[data[i].personId] = updateRatings[i]
    RDdict[data[i].personId] = updateRDs[i]
    voldict[data[i].personId] = updateVols[i]
end

1v: 0.08230665413221541
change: 45.142385355366265
new rating: NaN
new RD: NaN
new vol: NaN
2v: 0.07270617797728286
change: 32.64245555210984
new rating: NaN
new RD: NaN
new vol: NaN
3v: 0.07270617797728286
change: 31.82279822073389
new rating: NaN
new RD: NaN
new vol: NaN
4v: 0.07270617797728286
change: 31.003140889357947
new rating: NaN
new RD: NaN
new vol: NaN
5v: 0.07270617797728286
change: 30.183483557982004
new rating: NaN
new RD: NaN
new vol: NaN
6v: 0.07270617797728286
change: 29.36382622660606
new rating: NaN
new RD: NaN
new vol: NaN
7v: 0.07270617797728286
change: 28.54416889523012
new rating: NaN
new RD: NaN
new vol: NaN
8v: 0.07270617797728286
change: 27.724511563854175
new rating: NaN
new RD: NaN
new vol: NaN
9v: 0.07270617797728286
change: 26.904854232478232
new rating: NaN
new RD: NaN
new vol: NaN
10v: 0.07270617797728286
change: 26.08519690110229
new rating: NaN
new RD: NaN
new vol: NaN
11v: 0.07270617797728286
change: 25.265539569726347
new rating: NaN
new RD: NaN
new 

LoadError: [91mInterruptException:[39m

In [181]:
sort(collect(zip(values(ratingdict),keys(ratingdict))), rev = true)

116190-element Array{Tuple{Any,Any},1}:
 (1682.94, "2003ZBOR02")
 (1682.94, "2003ZBOR01")
 (1682.94, "2003WINT01")
 (1682.94, "2003WINO01")
 (1682.94, "2003WESS01")
 (1682.94, "2003WESL01")
 (1682.94, "2003WEBL01")
 (1682.94, "2003VAND01")
 (1682.94, "2003TREG02")
 (1682.94, "2003TREG01")
 (1682.94, "2003TING01")
 (1682.94, "2003THOM01")
 (1682.94, "2003TEMP01")
 ⋮                      
 (1500.0, "1982TENE01") 
 (1500.0, "1982SEBE01") 
 (1500.0, "1982SAND01") 
 (1500.0, "1982ROME01") 
 (1500.0, "1982RAZO01") 
 (1500.0, "1982LAET01") 
 (1500.0, "1982LABA01") 
 (1500.0, "1982JEAN01") 
 (1500.0, "1982GALR01") 
 (1500.0, "1982CHIL01") 
 (1500.0, "1982BRIN01") 
 (1500.0, "1982BORS01") 